In [1]:
import pandas as pd
import numpy as np
import os
from scipy.cluster import hierarchy as hc
import matplotlib
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from scipy import stats
import math
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
warnings.filterwarnings(action='ignore')

matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

# Load data

In [2]:
# dir_root = 'C:/Users/kkt71/OneDrive/바탕 화면/기업가치분석'
dir_root = 'C:/Users/10188/Desktop/기업가치/상관분석/기업가치분석_data_result'
dir_master = os.path.join(dir_root, '01. data', 'master')
dir_processed = os.path.join(dir_root, '01. data', 'processed')
dir_result = os.path.join(dir_root, '03. result')

In [3]:
y = '매출액증가율(YoY)(연도)'
c1 = 'Accenture'
c2 = 'Infosys'
c3 = 'TCS'
c4 = 'Cognizant'

In [4]:
outperform = [c1, c2, c3]
lowperform = [c4]

In [5]:
target_out = [s+'_show' for s in outperform]
target_low = [s+'_show' for s in lowperform]
target_col = [s+'_show' for s in outperform+lowperform]

In [6]:
dir_result_all = os.path.join(dir_result, 'All_ITS', y)

In [7]:
df = pd.read_excel(os.path.join(dir_result_all, 'corr_y_x_over7.xlsx'))

In [8]:
df.columns

Index(['variable', 'group', 'importance', 'Infosys_filtered', 'TCS_filtered',
       'Cognizant_filtered', 'Accenture', 'Infosys_show', 'TCS_show',
       'Cognizant_show'],
      dtype='object')

# Make index

In [9]:
df['Outperform_avg'] = df[list(set(target_out)&set(df.columns))].mean(axis=1)
df['Gap(Out~Low)'] = np.abs(df['Outperform_avg']-df[target_low[0]])
df['Std(Out)'] = df[list(set(target_out)&set(df.columns))].std(axis=1)

In [10]:
scaler = MinMaxScaler()
df['Gap normed+1'] = pd.DataFrame(scaler.fit_transform(df[['Gap(Out~Low)']])+1)[0]

In [11]:
scaler = MinMaxScaler()
df['Std normed+1'] = pd.DataFrame(scaler.fit_transform(df[['Std(Out)']])+1)[0]

In [12]:
df['index 1'] = df['Gap normed+1']/df['Std normed+1']

In [13]:
scaler = MinMaxScaler()
df['Std normed original'] = pd.DataFrame(scaler.fit_transform(df[['Std(Out)']]))[0]

In [14]:
df['Std normed 0.5'] = df['Std normed original']*0.5 + 0.5
df['1/Std normed'] = 1/df['Std normed 0.5']

In [15]:
df['Index 2'] = 0.6*df['Gap normed+1']+0.4*df['1/Std normed']

In [16]:
df[[s+'_marked' for s in outperform+lowperform]+[s+'_plot' for s in outperform+lowperform]] = np.nan

In [18]:
writer = pd.ExcelWriter(os.path.join(dir_result_all, 'correlation_index.xlsx'), engine='xlsxwriter')
df.to_excel(writer, index=False)
writer.save()

In [17]:
df

,variable,group,importance,Infosys_filtered,TCS_filtered,Cognizant_filtered,Accenture,Infosys_show,TCS_show,Cognizant_show,...,1/Std normed,Index 2,Accenture_marked,Infosys_marked,TCS_marked,Cognizant_marked,Accenture_plot,Infosys_plot,TCS_plot,Cognizant_plot
0,영업이익증가율(YoY)(연도),성장성,1,0.853,NaN,0.882,0.330,0.853,0.548,0.882,...,1.508885,1.245627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"BPS(지배, Adj.)(원/주)",안정성,2,0.843,NaN,0.855,0.302,0.843,0.281,0.855,...,1.249498,1.200148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,순이익증가율(YoY)(비율),성장성,1,0.827,NaN,NaN,0.259,0.827,0.615,0.163,...,1.631441,1.491427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ROE(지배)(비율),수익성,1,0.785,NaN,0.863,-0.064,0.785,0.689,0.863,...,1.815355,1.339208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FS_ROE,FS,5,0.785,NaN,0.863,-0.064,0.785,0.689,0.863,...,1.815355,1.339208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,FS_Gross Profit - Industrials/Property - Total,FS,5,NaN,NaN,-0.877,0.161,-0.626,-0.465,-0.877,...,1.707495,1.403469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,IS_Gross Profit,IS,5,NaN,NaN,-0.877,0.161,-0.675,-0.467,-0.877,...,1.637160,1.362007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,인당매출s,성장성,0,NaN,NaN,-0.885,0.008,0.440,0.288,-0.885,...,1.721659,1.888664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,"BS_Property/Plant/Equipment, Total - Gross",BS,5,NaN,NaN,-0.902,0.182,-0.608,-0.448,-0.902,...,1.709058,1.426306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
